<a href="https://colab.research.google.com/github/Jesimsim/ds-section2-project/blob/main/section2_project_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>